# <p style="text-align: center; font-style: strong;">Soft-NMS</p>
### <p style="text-align: center;">(Almond: 0.8.0, Scala: 2.12.8)</p>


## Dependencies

In [1]:
interp.load.ivy(coursierapi.Dependency.of("org.platanios", "tensorflow_2.12", "0.4.1").withClassifier("darwin-cpu-x86_64"))

In [2]:
import org.tensorflow.framework.GraphDef
import org.platanios.tensorflow.api.ops.{ Files, Image => TImage }
import org.platanios.tensorflow.api.core.client.{ FeedMap, Session }
import org.platanios.tensorflow.api.core.{ Graph, Shape, NewAxis }
import org.platanios.tensorflow.api.{ UByte, tf, Tensor, --- }
import java.io.{BufferedInputStream, File, FileInputStream}
import scala.io.Source
import scala.collection.mutable.ListBuffer
import scala.math.{min, max, exp, sqrt, abs}

import org.tensorflow.framework.GraphDef

import org.platanios.tensorflow.api.ops.{ Files, Image => TImage }

import org.platanios.tensorflow.api.core.client.{ FeedMap, Session }

import org.platanios.tensorflow.api.core.{ Graph, Shape, NewAxis }

import org.platanios.tensorflow.api.{ UByte, tf, Tensor, --- }

import java.io.{BufferedInputStream, File, FileInputStream}

import scala.io.Source

import scala.collection.mutable.ListBuffer

import scala.math.{min, max, exp, sqrt, abs}

## Initialization
Just modify *modelsFileName* and *imageName* and after respect the following project structure :

```
data    
└───labelsMap
   │   X.txt
   │   Y.txt
└───models
   │   X.pb
   │   Y.pb   
images
│   Z.txt
```


In [3]:
val basedir = "data"
val imageName = "0049.png"
val modelsFileName = List("coco", "kitti")
val imageFilePath = s"${System.getProperty("user.dir")}/images/${imageName}"

val labelsMapPath = modelsFileName.map(name => s"${System.getProperty("user.dir")}/${basedir}/labelsMap/${name}.txt")
val modelsGraphPath = modelsFileName.map(name => s"${System.getProperty("user.dir")}/${basedir}/models/${name}.pb")

basedir: String = "data"
imageName: String = "0049.png"
modelsFileName: List[String] = List("coco", "kitti")
imageFilePath: String = "/Users/vincentbrule/Desktop/notebook_project/notebookExamples/softNMS/images/0049.png"
labelsMapPath: List[String] = List(
  "/Users/vincentbrule/Desktop/notebook_project/notebookExamples/softNMS/data/labelsMap/coco.txt",
  "/Users/vincentbrule/Desktop/notebook_project/notebookExamples/softNMS/data/labelsMap/kitti.txt"
)
modelsGraphPath: List[String] = List(
  "/Users/vincentbrule/Desktop/notebook_project/notebookExamples/softNMS/data/models/coco.pb",
  "/Users/vincentbrule/Desktop/notebook_project/notebookExamples/softNMS/data/models/kitti.pb"
)

In [4]:
lazy val graphs = modelsGraphPath.map(model => Graph.fromGraphDef(GraphDef.parseFrom(new BufferedInputStream(new FileInputStream(new File(model))))))

graphs: List[Graph] = List(
  org.platanios.tensorflow.api.core.Graph@8801e553,
  org.platanios.tensorflow.api.core.Graph@8872f113
)

In [5]:
val sessions = graphs.map(Session(_))
val sessionSimple = Session()

sessions: List[Session] = List(
  org.platanios.tensorflow.api.core.client.Session@4a3b7dad,
  org.platanios.tensorflow.api.core.client.Session@329aceec
)
sessionSimple: Session = org.platanios.tensorflow.api.core.client.Session@23dd192f

## Prepare recuperation of graph results

In [6]:
val placeholdersOutput = graphs.map { graph => 
    Map("detectionBoxes" -> graph.getOutputByName("detection_boxes:0").toFloat,
        "detectionScores" -> graph.getOutputByName("detection_scores:0").toFloat,
        "detectionClasses" -> graph.getOutputByName("detection_classes:0").toFloat,
        "numDetections" -> graph.getOutputByName("num_detections:0").toFloat)
}

val placeholdersImage = graphs.map(_.getOutputByName("image_tensor:0").toUByte)

placeholdersOutput: List[Map[String, org.platanios.tensorflow.api.ops.Output[Float]]] = List(
  Map(
    "detectionBoxes" -> Output(detection_boxes, 0),
    "detectionScores" -> Output(detection_scores, 0),
    "detectionClasses" -> Output(detection_classes, 0),
    "numDetections" -> Output(num_detections, 0)
  ),
  Map(
    "detectionBoxes" -> Output(detection_boxes, 0),
    "detectionScores" -> Output(detection_scores, 0),
    "detectionClasses" -> Output(detection_classes, 0),
    "numDetections" -> Output(num_detections, 0)
  )
)
placeholdersImage: List[org.platanios.tensorflow.api.ops.Output[UByte]] = List(
  Output(image_tensor, 0),
  Output(image_tensor, 0)
)

## Open and transform image

In [7]:
val (imgTensor, fileNamePlaceholder) = tf.createWith() {
    val fileNamePlaceholder = tf.placeholder[String]()
    val fileTensor = Files.readFile(fileNamePlaceholder)
    val imgTensor = TImage.decodePng(fileTensor, 3)
    (imgTensor, fileNamePlaceholder)
  }

imgTensor: org.platanios.tensorflow.api.ops.Output[UByte] = Output(DecodePng, 0)
fileNamePlaceholder: org.platanios.tensorflow.api.ops.Output[String] = Output(
  Placeholder,
  0
)

In [8]:
val imageOuts = tf.createWith() {
    val file = new File(imageFilePath)
    val fileNameTensor = Tensor.fill(Shape())(file.getAbsolutePath())
    val feedImg = FeedMap(Map(fileNamePlaceholder -> fileNameTensor))
    sessionSimple.run(fetches = imgTensor, feeds = feedImg)
}

: 

In [ ]:
val listFeeds = placeholdersImage.map(placeholder => FeedMap(Map(placeholder -> imageOuts.slice(NewAxis, ---))))


## Read labelsMap file and transform it into Scala Map

In [ ]:
val labelsMapToString = labelsMapPath.map { path => 
    Source.fromFile(path).getLines.map { line =>
        val splitLine = line.split(" ")
        splitLine(0).toInt -> splitLine(1)
    }.toMap
}

## Detection 

In [ ]:
val threshold = 0.5 // Thereshold to discard detection below that
val height = imageOuts.shape(0)
val width = imageOuts.shape(1)
val sigma = 0.5

*boxes* = Positions of object detected

*score* = Confidence for each detection

*classes* = number corresponding to a class inside our labelMap previously defined

*num* = Number of detection

In [ ]:
val detections = sessions.zipWithIndex.map { case (s, index) =>
      s.run(
        fetches =
          Seq(placeholdersOutput(index)("detectionBoxes"), placeholdersOutput(index)("detectionScores"), placeholdersOutput(index)("detectionClasses"), placeholdersOutput(index)("numDetections")),
          feeds = listFeeds(index))
}

## Boxes filter

In [ ]:
// Use relative positions to be able to draw bounding box after
val listTabBoxes = detections.zipWithIndex.map { case (detection, index) => 
    for {
        i <- 0 until detection(3).scalar.toInt
        labelId = detection(2)(0, i).toFloat.scalar.toInt
        box = detection(0)(0, i).toFloat.entriesIterator.toSeq
        y1 = (box(0))
        x1 = (box(1))
        y2 = (box(2))
        x2 = (box(3))
        labelBox = List(y1, x1, y2, x2)
        score = detection(1)(0, i).toFloat.scalar
  } yield (labelsMapToString(index)(labelId), score, labelBox)
}

## Scala boxes to Tensorflow boxes

In [ ]:
// We start by keep only positions and flatMap all boxes and transform it into Tensor
val listFlatBoxes = listTabBoxes.map { tabBoxesFiltered => Tensor(tabBoxesFiltered.flatMap{ case(_, _, positions) => positions }) }
// We reshape it to have shape like : [batch, num_bounding_boxes, 4]
val listTensorWithAllBoxes = listFlatBoxes.map { tensorFlatBoxes => tf.reshape(tensorFlatBoxes, Shape(1, tensorFlatBoxes.shape(1) / 4, 4)) }


In [ ]:
// We combined detections boxes from all models
val boxesCombined = Tensor(listTabBoxes.flatMap {boxes => boxes.flatMap { case(_, _, positions) => positions } })
// We obtain a Tensor with Shape [1, num_all_boxes, 4]
val tensorBoxesCombined = tf.reshape(boxesCombined, Shape(1, listTabBoxes.foldLeft(0) { (acc, i) => acc + i.length }, 4))

## Display results

In [ ]:
// list with image + detection boxes for each model
val listImageToDisplay = listTensorWithAllBoxes.map { tensorWithAllBoxes =>
    tf.reshape(TImage.drawBoundingBoxes(imageOuts.slice(NewAxis, ---).toFloat, tensorWithAllBoxes).toUByte, Shape(height, width, 3))
}

// image with all boxes
val imageCombinedToDisplay = tf.reshape(TImage.drawBoundingBoxes(imageOuts.slice(NewAxis, ---).toFloat, tensorBoxesCombined).toUByte, Shape(height, width, 3))

In [ ]:
val listImgFinal = listImageToDisplay.map { image => 
    tf.createWith() {
        val exampleImage = tf.decodeRaw[Byte](tf.image.encodePng(image))
        sessionSimple.run(fetches = exampleImage)
    }
}

val imgFinalCombined = tf.createWith() {
    val exampleImage = tf.decodeRaw[Byte](tf.image.encodePng(imageCombinedToDisplay))
    sessionSimple.run(fetches = exampleImage)
}

## Display result

In [ ]:
listImgFinal.foreach { image => Image(image.entriesIterator.toArray).withFormat(Image.PNG).withWidth(500).display }

In [ ]:
Image(imgFinalCombined.entriesIterator.toArray).withFormat(Image.PNG).withWidth(500).display

## Soft-NMS

In [ ]:
// Group identification by class because Soft-NMS works for each class separately
val mapDetections = listTabBoxes.flatten.map { case(s, c, l) => (s, c, List(l(0)*height, l(1)*width, l(2)*height, l(3)*width)) }.groupBy { case(c, _, _) => c }
// Keep only car and person detections
var listBoxesCar = mapDetections("car").to[ListBuffer]
var listBoxesPerson = mapDetections("person").to[ListBuffer]

## Implementation transformed into more functional way
Below this, you can find the python copied implementation

In [ ]:
def softNMS(boxesWithScore: ListBuffer[(String, Float, List[Float])]): List[List[Float]] = {
    var listInitial = boxesWithScore.clone()
    var listFinalDetectionBoxes = ListBuffer[List[Float]]()
    
    while (listInitial.nonEmpty) {
        // Find box with the max score detection
        val maxBox = listInitial.maxBy(_._2)
        val List(ty1, tx1, ty2, tx2) = maxBox._3
        listFinalDetectionBoxes += listInitial.remove(listInitial.indexOf(maxBox))._3
        // Modify score in function of the shared area
        listInitial = listInitial.map {
            case (c, s, p) => 
                val area = (p(3) - p(1) + 1) * (p(2) - p(0) + 1)
                val iw = (min(tx2, p(3)) - max(tx1, p(1)) + 1)
                val ih = (min(ty2, p(2)) - max(ty1, p(0)) + 1)
                val scoreWeighted = 
                    if (iw > 0 && ih > 0) {
                        val ua = ((tx2 - tx1 + 1) * (ty2 - ty1 + 1) + area - iw * ih)
                        val ov = iw * ih / ua
                        s * exp(-(ov * ov) / sigma).toFloat
                    }
                    else s
                (c, scoreWeighted, p)
        }
        // Remove boxes in function of our threshold and their nms score
        listInitial = listInitial.filterNot(_._2 < threshold)
    }
    
    listFinalDetectionBoxes.toList
}

## Implementation copied from Python Code

## Display results

In [ ]:
val nmsBoxes = softNMS(listBoxesCar).flatMap { case(List(y1,x1,y2,x2)) => List(y1/height, x1/width, y2/height, x2/width)}


In [ ]:
val tensorNmsBoxes = Tensor(nmsBoxes)
val tensorReshapeNmsBoxes = tf.reshape(tensorNmsBoxes, Shape(1, nmsBoxes.length / 4, 4))
val imageCombinedNms = tf.reshape(TImage.drawBoundingBoxes(imageOuts.slice(NewAxis, ---).toFloat, tensorReshapeNmsBoxes).toUByte, Shape(height, width, 3))
val imgFinalCombinedNms = tf.createWith() {
    val exampleImage = tf.decodeRaw[Byte](tf.image.encodePng(imageCombinedNms))
    sessionSimple.run(fetches = exampleImage)
}
Image(imgFinalCombinedNms.entriesIterator.toArray).withFormat(Image.PNG).withWidth(500).display
